In [10]:
import pandas as pd
import numpy as np

def topsis_analysis(df, criteria_columns, weights, impacts):
    """
    Perform TOPSIS analysis on the dataset
    
    Parameters:
    - df: Input DataFrame
    - criteria_columns: List of columns to use for TOPSIS
    - weights: List of weights for each criterion
    - impacts: List of impact directions ('+' for maximize, '-' for minimize)
    
    Returns:
    - DataFrame with TOPSIS scores and ranks
    """
    # Create a copy of the dataframe
    data = df.copy()
    
    # Normalize the decision matrix
    normalized_matrix = data[criteria_columns].copy()
    for col in criteria_columns:
        # Normalize using vector normalization
        normalized_matrix[col] = normalized_matrix[col] / np.sqrt((normalized_matrix[col]**2).sum())
    
    # Apply weights
    weighted_matrix = normalized_matrix.copy()
    for i, col in enumerate(criteria_columns):
        weighted_matrix[col] *= weights[i]
    
    # Determine ideal best and worst solutions
    ideal_best = []
    ideal_worst = []
    for i, col in enumerate(criteria_columns):
        if impacts[i] == '+':
            ideal_best.append(weighted_matrix[col].max())
            ideal_worst.append(weighted_matrix[col].min())
        else:
            ideal_best.append(weighted_matrix[col].min())
            ideal_worst.append(weighted_matrix[col].max())

    # Calculate distances to ideal best and worst solutions
    distance_best = np.sqrt(((weighted_matrix[criteria_columns] - ideal_best)**2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_matrix[criteria_columns] - ideal_worst)**2).sum(axis=1))
    
    # Calculate TOPSIS score
    data['Topsis_Score'] = distance_worst / (distance_best + distance_worst)
    
    # Rank the alternatives
    data['Rank'] = data['Topsis_Score'].rank(method='dense', ascending=False)
    
    return data

for YEAR in range(2023, 2039):
    # Read the dataset
    df = pd.read_csv(f'data/topsis_input_{YEAR}.csv', index_col=False)
    df.reset_index(drop=True, inplace=True)
    df['Total_Cost'] = df['insurance_cost'] + df['maintenance_cost'] + df['fuel_costs_per_km'] + df['Cost ($)']

    # Get unique combinations of Size and Distance_x
    combinations = df.groupby(['Size', 'Distance_demand'])

    # Store results
    results = []

    # Perform TOPSIS for each combination
    for (size, distance), group in combinations:
        # Define criteria columns
        criteria_columns = ['carbon_emissions_per_km','Total_Cost']
        
        # Define weights (equal weights in this case)
        weights = [0.5, 0.5]
        
        # Define impact directions (both to be minimized)
        impacts = ['-', '-']
        
        # Perform TOPSIS
        result = topsis_analysis(group, criteria_columns, weights, impacts)
        
        # Add size and distance info to results
        result['Size'] = size
        result['Distance'] = distance
        
        results.append(result)

    # Combine all results
    final_results = pd.concat(results)

    # Save to CSV
    final_results.to_csv(f'topsis_results_{YEAR}.csv', index=False)

    print(f"TOPSIS analysis for {YEAR} completed. Results saved to topsis_results_{YEAR}.csv")

TOPSIS analysis for 2023 completed. Results saved to topsis_results_2023.csv
TOPSIS analysis for 2024 completed. Results saved to topsis_results_2024.csv
TOPSIS analysis for 2025 completed. Results saved to topsis_results_2025.csv
TOPSIS analysis for 2026 completed. Results saved to topsis_results_2026.csv
TOPSIS analysis for 2027 completed. Results saved to topsis_results_2027.csv
TOPSIS analysis for 2028 completed. Results saved to topsis_results_2028.csv
TOPSIS analysis for 2029 completed. Results saved to topsis_results_2029.csv
TOPSIS analysis for 2030 completed. Results saved to topsis_results_2030.csv
TOPSIS analysis for 2031 completed. Results saved to topsis_results_2031.csv
TOPSIS analysis for 2032 completed. Results saved to topsis_results_2032.csv
TOPSIS analysis for 2033 completed. Results saved to topsis_results_2033.csv
TOPSIS analysis for 2034 completed. Results saved to topsis_results_2034.csv
TOPSIS analysis for 2035 completed. Results saved to topsis_results_2035.csv

In [ ]:
import pandas as pd

dfs = []
for YEAR in range(2023, 2039):
    df = pd.read_csv(f'topsis_results_{YEAR}.csv', index_col=False)
    top_ranked = df[df["Rank"] == 1].copy()
    
    # Add a column to indicate the year (if needed for reference)
    top_ranked["Year"] = YEAR
    
    # Append the filtered DataFrame to the list
    dfs.append(top_ranked)

# Combine all filtered DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
final_df.to_csv('top_ranked_combined.csv', index=False)

print("CSV files processed and combined successfully!")


,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km,insurance_cost,maintenance_cost,fuel_costs_per_km,Total_Cost,Topsis_Score,Rank,Distance
0,BEV_S1_2038,2038,S1,D1,1254358,BEV_S1_2038,BEV,2038,130589,102000,D4,Electricity,0.000000,6529.45,1305.89,0.090210,7835.430210,1.000000,1.0,D1
1,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,B20,0.679882,6621.35,1324.27,0.373291,7945.993291,0.078900,5.0,D1
2,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,HVO,0.108748,6621.35,1324.27,0.343929,7945.963929,0.840623,3.0,D1
3,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,LNG,0.384701,7789.80,1557.96,0.143362,9347.903362,0.430935,4.0,D1
4,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,BioLNG,0.058424,7789.80,1557.96,0.153808,9347.913808,0.878690,2.0,D1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,BEV_S4_2038,2038,S4,D4,2446,BEV_S4_2038,BEV,2038,276287,118000,D4,Electricity,0.000000,13814.35,2762.87,0.088993,16577.308993,0.927751,1.0,D4
56,Diesel_S4_2038,2038,S4,D4,2446,Diesel_S4_2038,Diesel,2038,235677,118000,D4,B20,0.670160,11783.85,2356.77,0.367953,14140.987953,0.112040,5.0,D4
57,Diesel_S4_2038,2038,S4,D4,2446,Diesel_S4_2038,Diesel,2038,235677,118000,D4,HVO,0.107193,11783.85,2356.77,0.339011,14140.959011,0.841542,3.0,D4
58,LNG_S4_2038,2038,S4,D4,2446,LNG_S4_2038,LNG,2038,301395,118000,D4,LNG,0.383564,15069.75,3013.95,0.142938,18083.842938,0.421855,4.0,D4
